## sub-task 3 제출 스크립트
ETRI FASHION-HOW Season 5 task3 제출코드입니다.
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.
- 작성하신 추론용 코드(예: main.py)를 본 스크립트 내의 etri_task3_submit() 함수로 작동되게끔 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
- **Continual learning**을 평가하는 과제로, 6개 데이터셋을 **순차적으로 학습**한 모델을 업로드하여 각 데이터에 대해 추론을 수행하도록 구성합니다.


코드는 크게 4가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치
2. etri_task3_submit() 함수 편집 (추론 스크립트)
3. submit() 함수로 wrapping
4. if __name__ == "__main__" 조건문을 이용한 제출 수행

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

### 2. etri_task3_submit() 함수 편집 (추론 스크립트)
#### etri_task3_submit() 편집 시 주의사항

1. 아래 etri_task3_submit() 함수 내에 전체 추론 실행 코드를 삽입, 6개 테스트셋에 대하여 추론을 6회 수행하고 결과를 list로 묶어 return하게끔 구성
   - Baseline이 아닌 다른 모델을 사용하는 경우에도 동일
2. 함수 내에서는 import * 가 적용되지 않으므로 필요한 import object를 직접 입력
   - 반드시 함수 내에서 import가 이루어져야 합니다.
3. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
4. 모델 내부의 경로는 ./ 으로 경로를 지정합니다. (예: weight 파일 경로 = ./model/...)
5. 데이터는 **/aif/data/** 경로 아래에 있습니다. (코드 내용 참조)
   - subWordEmb_path는 /aif/sstm_v0p5_deploy/ 안에 .dat 파일이 있습니다.
6. return할 결과물과 양식에 유의합니다.

In [2]:
# main.py의 내용을 test 옵션과 제출 후 자동추론 환경에 맞게끔 수정하여 함수화

def etri_task3_submit():
    # torch
    import torch

    # built-in library
    import os
    import argparse

    # custom modules
    from exp_utils import get_udevice, load_cfg
    from exp_manager import Manager
    
    cores = os.cpu_count()
    torch.set_num_threads(cores)

    ### input options ###
    parser = argparse.ArgumentParser(description='AI Fashion Coordinator.')

    parser.add_argument('--cfg_path', type=str,
                        default='./cfgs/09_dec_model_size_with_08_masks.yaml', # 주의) 모델 변경 시 cfg 파일 경로도 변경
                        help="실험에 필요한 값들을 설정해둔 yaml 파일의 경로를 입력합니다.")

    args, _ = parser.parse_known_args()
    ### input options ###

    # load configuration file
    cfg = load_cfg(args.cfg_path)

    # set pred mode
    cfg['global']['mode'] = 'pred'

    # check configuration value
    print('<Parsed arguments>')
    for category, value in cfg.items():
        print(f"##### {category} #####")
        for name, value in cfg[category].items():
            print(f"{name}: {value}")

        print('-' * 20)

    # set model
    manager = Manager(cfg, get_udevice())
    
    ####### 테스트 파일 경로를 바꿔가면서 6회의 로드와 추론을 수행 #######
    ### 다른 모델을 사용하는 경우에도 6회의 추론이 수행되어야 함에 유의 ###

    out = [] # 각 예측 결과를 리스트 형태로 묶어서 리턴
    
    for i in range(6):
        manager._in_file_tst_dialog = f'/aif/data/cl_eval_task{i + 1}.wst.tst.2024' # 경로에 주의
        out.append(manager.pred(task_id=i))
    
    return out 

### 3. submit() 함수로 wrapping
#### 반드시 아래와 같이 submit() 이라는 함수로 위에 정의된 etri_task3_submit 함수를 wrapping해야 합니다.

In [3]:
def submit():
    return etri_task3_submit

### 4. if name == "main" 조건문을 이용한 제출 수행
#### 아래와 같이 if __name__ == "__main__" 구문 내에서 제출 함수가 실행되게끔 합니다.
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 할당된 계정을 이용하여 task별로 확인하실 수 있습니다.

In [4]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="09_dec_model_size_with_08_masks",
               key="a07806dd-83c3-451e-82dd-cabaa0668b9c",
               func=submit # submit function
               )
    #-----------------------------------------------------#
    print(time.time() - t)

file : task.py
python
파일 전송 완료
score = 345.7167(Success - Your WKT score: 0.7151515, Weight Size: 345.7167)
155.333425283432
